In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os

In [7]:
# Reading "gubra_allen_map.csv" file
brain_gubra_map = pd.read_csv("brain_gubra_map.csv", 
                               dtype={"IDPath": "string",
                                      "LabelAbrv": "string",
                                      "raw": "string"})

In [15]:
#Creating the 'LabelID' column
brain_gubra_map['LabelID'] = brain_gubra_map['IDPath'].str.split('/').str[-2]

In [24]:
#Creating the right gruba intensities template
right_gubra = brain_gubra_map[['IDPath','LabelID','raw','LabelAbrv']]
right_gubra[['LabelID']] = right_gubra[['LabelID']].astype(int)


In [31]:
#Creating the left gruba intensities template
left_gubra = right_gubra
left_gubra


,IDPath,LabelID,raw,LabelAbrv
0,/997/8/567/623/477/278/23/,23,Anterior amygdalar area,AAA
1,/997/8/567/688/695/315/31/39/935/,935,"Anterior cingulate area, dorsal part, layer 1",ACAd1
2,/997/8/567/688/695/315/31/39/211/,211,"Anterior cingulate area, dorsal part, layer 2/3",ACAd2/3
3,/997/8/567/688/695/315/31/39/1015/,1015,"Anterior cingulate area, dorsal part, layer 5",ACAd5
4,/997/8/567/688/695/315/31/39/919/,919,"Anterior cingulate area, dorsal part, layer 6a",ACAd6a
...,...,...,...,...
588,/997/1009/967/933/413/,413,vestibular nerve,vVIIIn
589,/997/8/343/1065/354/370/765/,765,Nucleus x,x
590,/997/8/343/1065/354/370/773/,773,Hypoglossal nucleus,XII
591,/997/8/343/1065/354/370/781/,781,Nucleus y,y


In [ ]:
#Saving drugs and markers' names from the "sample_overview" file

drugs = samples_overview['TX_GROUP'].unique()
markers = samples_overview['marker'].unique()

d = 0
m = 0
counters_names = []
while (m < len(markers)):
    while (d < len(drugs)):
        counters_names.append(drugs[d] + '_' + markers[m])
        d += 1
    m += 1
    d = 0

In [ ]:
# Changing directory to the "02Volume_files" folder
os.chdir(subfolder_with_path[1])

#Counting number of files on "02Volume_files" folder
list = os.listdir(subfolder_with_path[1]) 
number_files = len(list)
number_volume_files = number_files

In [ ]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_data = [] #This array will contain df as elements

i = 0
while i < number_files:

    ##############################################
    # Processing each file
    ##############################################
    
    file = list[i]
    # Saving sample's number
    filename_segmented = file.split('_')
    sample = filename_segmented[1]


    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME'] = output['VOLUME'].div(1000000).round(7)
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    else:
        output = output[output['LabelID'] < 20000]
   

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["labelabrv"] = output["labelabrv"].str.replace('L','',1)
    output["labelabrv"] = output["labelabrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
volume_data = pd.concat(volume_data)

In [ ]:
# Changing directory to the "03Volume_in_tissue_files" folder
os.chdir(subfolder_with_path[2])

#Counting number of files on "03Volume_in_tissue_files" folder
list = os.listdir(subfolder_with_path[2]) 
number_files = len(list)
number_volintissue_files = number_files

In [ ]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_intissue_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter


#Zeroing the counters 
counters = np.zeros(len(counters_names))

#Creating counters array to contain sample's names as elements
counters_samples = [[] for j in range(len(counters_names))]

    ##############################################
    # Processing each file
    ##############################################

while i < number_files:


    file = list[i]
    # Saving sample's number 
    filename_segmented = file.split('_')
    sample = filename_segmented[1]

    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME_IN_TISSUE'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME_IN_TISSUE'] = output['VOLUME_IN_TISSUE'].div(1000000).round(6)
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    else:
        output = output[output['LabelID'] < 20000]

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE','GENDER',
                                             'hemisphere','marker',
                                             'TX_GROUP','CONTEXT']],on='SAMPLE',how='inner')
    
    #Adding a column with the sample's nickname

    counters_index = counters_names.index(output['TX_GROUP'].iloc[0] + '_' + output['marker'].iloc[0])
    if output['SAMPLE'].iloc[0] not in counters_samples[counters_index]:
        counters[counters_index] += 1
        counters_samples[counters_index].append(output['SAMPLE'].iloc[0])
        nickname = 'v' + output['TX_GROUP'].iloc[0] + '_' + str(int(counters[counters_index]))

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname
    
    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["labelabrv"] = output["labelabrv"].str.replace('L','',1)
    output["labelabrv"] = output["labelabrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_intissue_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframeo
volume_intissue_data = pd.concat(volume_intissue_data)

In [ ]:
#calculating volume ratio
volumes_data = pd.merge(volume_intissue_data,volume_data[['raw','VOLUME','SAMPLE']],on=('raw','SAMPLE'),how='left')
volumes_data['volumes_ratio'] = volumes_data['VOLUME_IN_TISSUE'] / volumes_data['VOLUME'] 

In [ ]:
#Pivoting dataframe to have nicknames as columns
pivoted_volumes_data = pd.pivot_table(volumes_data, values='VOLUME_IN_TISSUE', 
                                       index=['raw'],columns=['nickname'], aggfunc=np.sum)

In [ ]:
# Changing directory to the "01Activity_files" folder
os.chdir(subfolder_with_path[0])

#Counting number of files on "01Activity_files" folder
list = os.listdir(subfolder_with_path[0]) 
number_files = len(list)
number_activity_files = number_files

In [ ]:
#Using and timing  a loop to process all files and populate the main data-frame

activity_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

#Zeroing the counters 
counters = np.zeros(len(counters_names))

#Creating counters array to contain sample's names as elements
counters_samples = [[] for j in range(len(counters_names))]


    ##############################################
    # Innitiates loop to process each file
    ##############################################


while i < number_files:

    # Working with each file at a time
    file = list[i]
    
    # Saving sample's number
    filename_segmented = file.split('_')
    sample = filename_segmented[1]

    # Reading file's data
    regions_count_activity_fracc = pd.read_csv(file,low_memory=False)
    regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"]        

    # Adding-up regions' fracctions
    regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
    regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
    regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
    regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
    regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
    regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})
    total_region_activity = regions_fracc_1.append(regions_fracc_2, 
                                                   ignore_index=True).append(regions_fracc_3, ignore_index=True)
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
    total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
#     total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)
    total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)
        
    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    activity_label_name = 'ACTIVITY'
    output = output.rename(columns = {'COUNTS':activity_label_name})
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    else:
        output = output[output['LabelID'] < 20000]

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE','GENDER',
                                             'hemisphere','marker',
                                             'TX_GROUP','CONTEXT']],on='SAMPLE',how='inner')
    
    
    #Adding a column with the sample's nickname

    counters_index = counters_names.index(output['TX_GROUP'].iloc[0] + '_' + output['marker'].iloc[0])
    if output['SAMPLE'].iloc[0] not in counters_samples[counters_index]:
        counters[counters_index] += 1
        counters_samples[counters_index].append(output['SAMPLE'].iloc[0])
        nickname = output['TX_GROUP'].iloc[0] + '_' + str(int(counters[counters_index]))

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname

    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["labelabrv"] = output["labelabrv"].str.replace('L','',1)
    output["labelabrv"] = output["labelabrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    activity_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
activity_data = pd.concat(activity_data)

In [ ]:
#Replacing empty cells with zeros
activity_data['ACTIVITY'].fillna(0, inplace=True)

In [ ]:
#Creating density dataframe
##Merging activity, volume and volume in tissue dataframes
density = pd.merge(activity_data,
                                  volumes_data[['raw','SAMPLE','VOLUME']],
                                  on=['raw','SAMPLE'],
                                  how='left')
density = pd.merge(density,
                           volume_intissue_data[['raw','SAMPLE','VOLUME_IN_TISSUE']],
                           on=['raw','SAMPLE'],
                           how='left')
#Creating the density nickname (d_nickname)
density['DENSITY'] = density['ACTIVITY'] / density['VOLUME_IN_TISSUE']
density['d_nickname'] = 'd_' + density['nickname']


#Pivoting dataframe to have d_nicknames as columns
pivoted_density = pd.pivot_table(density, values='DENSITY', 
                                       index=['raw'],columns=['d_nickname'], aggfunc=np.sum)

In [ ]:
#Merging activity and volume ratio to filter activity where the volume ratio is below 50%
filtered_activity_data = pd.merge(activity_data,
                                  volumes_data[['raw','SAMPLE','volumes_ratio']],
                                  on=['raw','SAMPLE'],
                                  how='left')
filtered_activity_data.loc[(filtered_activity_data['volumes_ratio'] < 0.5),'ACTIVITY'] = ''

In [ ]:
#Pivoting dataframe to have nicknames as columns
pivoted_activity_data = pd.pivot_table(filtered_activity_data, values='ACTIVITY', 
                               index=['raw','hemisphere','marker'],columns=['nickname'], aggfunc=np.sum)

#Restarting index to activate'hemisphere','marker'and 'rater' as columns
pivoted_activity_data = pivoted_activity_data.reset_index()

In [ ]:
#combining all dataframes
allen_and_activity = pd.merge(brain_allen_map,pivoted_activity_data,on='raw',how='right')
cell_count_output = pd.merge(allen_and_activity,pivoted_volumes_data,on='raw',how='left')
cell_count_output = pd.merge(cell_count_output,pivoted_density,on='raw',how='left')

In [ ]:
# Changing directory to the main path
os.chdir(main_directory)

In [ ]:
#creating the "cell_count_output.csv" file
cell_count_output.to_csv('ALLEN_cell_count_output.csv', index=False)

In [ ]:
#Ending timming process
end = time.time()

# Printing how many files were processed and how much time the process took
files_processed = number_activity_files = number_files+ number_volume_files + number_volintissue_files
print(files_processed,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 